# TPC-H Queries

In this notebook, we explore how to reimplement SQL queries from the well-known TPC-H benchmark in Rabbit.

We start with loading the dataset.

In [1]:
include("../tpch.jl")

using RBT
setdb(tpch)

Region:
  name :: ASCIIString
  comment :: ASCIIString
  nation (inverse of Nation.region) :: Array{Nation,1} # unique, covering
Nation:
  name :: ASCIIString
  region :: Region
  comment :: ASCIIString
  customer (inverse of Customer.nation) :: Array{Customer,1} # unique, covering
  supplier (inverse of Supplier.nation) :: Array{Supplier,1} # unique, covering
Customer:
  name :: ASCIIString
  address :: ASCIIString
  nation :: Nation
  phone :: ASCIIString
  acctbal :: RBT.Monetary{:USD}
  mktsegment :: ASCIIString
  comment :: ASCIIString
  order (inverse of Order.customer) :: Array{Order,1} # unique, covering
Supplier:
  name :: ASCIIString
  address :: ASCIIString
  nation :: Nation
  phone :: ASCIIString
  acctbal :: RBT.Monetary{:USD}
  comment :: ASCIIString
  partsupp (inverse of Partsupp.supplier) :: Array{Partsupp,1} # unique, covering
  lineitem (inverse of Lineitem.supplier) :: Array{Lineitem,1} # unique, covering
Part:
  name :: ASCIIString
  mfgr :: ASCIIString
  brand ::

The TPC-H benchmark consists of a generated dataset of customers, suppliers, products and orders as well as a collection of queries that perform various types of business analysis.

Table `order` lists all orders made by customers.

In [2]:
@query(
    order
    :select(
        customer.name, totalprice, orderdate))

,name,totalprice,orderdate
1,Customer#000000296,159877.96,1996-01-02
2,Customer#000000625,69858.91,1996-12-01
3,Customer#000000988,183718.98,1993-10-14
4,Customer#000001096,50463.93,1995-10-11
5,Customer#000000356,133409.48,1994-07-30
6,Customer#000000445,35731.21,1992-02-21
7,Customer#000000314,229881.30,1996-01-10
8,Customer#000001042,173503.97,1995-07-16
9,Customer#000000536,135080.42,1993-10-27
10,Customer#000000490,64891.67,1998-07-21


Individual items in each order are stored in the `lineitem` table.  Each item lists the ordered part, the supplier and the pricing information.

In [3]:
@query(
    lineitem
    :select(
        order.customer, linenumber, part, supplier, quantity, extendedprice, discount, tax, shipdate))

,customer,linenumber,part,supplier,quantity,extendedprice,discount,tax,shipdate
1,Customer#000000296,1,dodger brown ivory blue pink,Supplier#000000078,17,19435.08,0.04,0.02,1996-03-13
2,Customer#000000296,2,dodger midnight salmon drab saddle,Supplier#000000060,36,51823.08,0.09,0.06,1996-04-12
3,Customer#000000296,3,blush thistle orchid red lace,Supplier#000000031,8,11284.08,0.1,0.02,1996-01-29
4,Customer#000000296,4,turquoise indian lemon lavender misty,Supplier#000000039,28,25704.28,0.09,0.06,1996-04-21
5,Customer#000000296,5,turquoise lime royal metallic azure,Supplier#000000020,24,26236.56,0.1,0.04,1996-03-30
6,Customer#000000296,6,burnished black blue metallic orchid,Supplier#000000009,32,32835.84,0.07,0.02,1996-01-30
7,Customer#000000625,1,peach goldenrod honeydew moccasin sienna,Supplier#000000031,38,66532.30,0.0,0.05,1997-01-28
8,Customer#000000988,1,green blush tomato burlywood seashell,Supplier#000000016,45,42076.35,0.06,0.0,1994-02-02
9,Customer#000000988,2,linen frosted slate coral peru,Supplier#000000057,49,51604.35,0.1,0.0,1993-11-09
10,Customer#000000988,3,blue drab lime khaki sienna,Supplier#000000053,27,25083.54,0.06,0.07,1994-01-16


For each customer and supplier, we track their country of origin.  Table `nation` contains a list of all countries partitioned by region.

In [4]:
@query(
    nation
    :select(
        name,
        region,
        cust_count => count(customer),
        supp_count => count(supplier)))

,name,region,cust_count,supp_count
1,ALGERIA,AFRICA,51,3
2,ARGENTINA,AMERICA,50,3
3,BRAZIL,AMERICA,63,1
4,CANADA,AMERICA,48,2
5,EGYPT,MIDDLE EAST,56,4
6,ETHIOPIA,AFRICA,44,3
7,FRANCE,EUROPE,32,1
8,GERMANY,EUROPE,41,4
9,INDIA,ASIA,52,5
10,INDONESIA,ASIA,56,4


The following query descriptions are taken directly from TPC-H documentation.  We will show how to incrementally construct these queries in Rabbit.

## Pricing Summary Report Query (Q1)
This query reports the amount of business that was billed, shipped, and returned.

### Business Question
The Pricing Summary Report Query provides a summary pricing report for all lineitems shipped as of a given date.
The date is within 60 - 120 days of the greatest ship date contained in the database. The query lists totals for
extended price, discounted extended price, discounted extended price plus tax, average quantity, average extended
price, and average discount. These aggregates are grouped by RETURNFLAG and LINESTATUS, and listed in
ascending order of RETURNFLAG and LINESTATUS. A count of the number of lineitems in each group is
included.
        
### Query Definition
```
select
    l_returnflag,
    l_linestatus,
    sum(l_quantity) as sum_qty,
    sum(l_extendedprice) as sum_base_price,
    sum(l_extendedprice*(1-l_discount)) as sum_disc_price,
    sum(l_extendedprice*(1-l_discount)*(1+l_tax)) as sum_charge,
    avg(l_quantity) as avg_qty,
    avg(l_extendedprice) as avg_price,
    avg(l_discount) as avg_disc,
    count(*) as count_order
from
    lineitem
where
    l_shipdate <= date '1998-12-01' - interval '[DELTA]' day
group by
    l_returnflag,
    l_linestatus
order by
    l_returnflag,
    l_linestatus;
```

### Substitution Parameters
1. DELTA = 90.

To generate this report, we use the data from the `lineitem` table.

In [5]:
@query(
    lineitem
    :select(order, linenumber, shipdate, quantity, extendedprice, discount, tax, returnflag, linestatus))

,order,linenumber,shipdate,quantity,extendedprice,discount,tax,returnflag,linestatus
1,Customer#000000296,1,1996-03-13,17,19435.08,0.04,0.02,N,O
2,Customer#000000296,2,1996-04-12,36,51823.08,0.09,0.06,N,O
3,Customer#000000296,3,1996-01-29,8,11284.08,0.1,0.02,N,O
4,Customer#000000296,4,1996-04-21,28,25704.28,0.09,0.06,N,O
5,Customer#000000296,5,1996-03-30,24,26236.56,0.1,0.04,N,O
6,Customer#000000296,6,1996-01-30,32,32835.84,0.07,0.02,N,O
7,Customer#000000625,1,1997-01-28,38,66532.30,0.0,0.05,N,O
8,Customer#000000988,1,1994-02-02,45,42076.35,0.06,0.0,R,F
9,Customer#000000988,2,1993-11-09,49,51604.35,0.1,0.0,R,F
10,Customer#000000988,3,1994-01-16,27,25083.54,0.06,0.07,A,F


We start with extracting all lineitems in the selected date range.

In [6]:
@query(
    lineitem
    :filter(shipdate <= date("1998-12-01") - DELTA*days),
    DELTA=90)

,order,part,supplier,linenumber,quantity,extendedprice,discount,tax,returnflag,linestatus,shipdate,commitdate,receiptdate,shipinstruct,shipmode,comment
1,Customer#000000296,dodger brown ivory blue pink,Supplier#000000078,1,17,19435.08,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER IN PERSON,TRUCK,egular courts above the
2,Customer#000000296,dodger midnight salmon drab saddle,Supplier#000000060,2,36,51823.08,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE BACK RETURN,MAIL,ly final dependencies: slyly bold
3,Customer#000000296,blush thistle orchid red lace,Supplier#000000031,3,8,11284.08,0.1,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE BACK RETURN,REG AIR,"riously. regular, express dep"
4,Customer#000000296,turquoise indian lemon lavender misty,Supplier#000000039,4,28,25704.28,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR,lites. fluffily even de
5,Customer#000000296,turquoise lime royal metallic azure,Supplier#000000020,5,24,26236.56,0.1,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB,pending foxes. slyly re
6,Customer#000000296,burnished black blue metallic orchid,Supplier#000000009,6,32,32835.84,0.07,0.02,N,O,1996-01-30,1996-02-07,1996-02-03,DELIVER IN PERSON,MAIL,arefully slyly ex
7,Customer#000000625,peach goldenrod honeydew moccasin sienna,Supplier#000000031,1,38,66532.30,0.0,0.05,N,O,1997-01-28,1997-01-14,1997-02-02,TAKE BACK RETURN,RAIL,ven requests. deposits breach a
8,Customer#000000988,green blush tomato burlywood seashell,Supplier#000000016,1,45,42076.35,0.06,0.0,R,F,1994-02-02,1994-01-04,1994-02-23,NONE,AIR,ongside of the furiously brave acco
9,Customer#000000988,linen frosted slate coral peru,Supplier#000000057,2,49,51604.35,0.1,0.0,R,F,1993-11-09,1993-12-20,1993-11-24,TAKE BACK RETURN,RAIL,unusual accounts. eve
10,Customer#000000988,blue drab lime khaki sienna,Supplier#000000053,3,27,25083.54,0.06,0.07,A,F,1994-01-16,1993-11-22,1994-01-23,DELIVER IN PERSON,SHIP,nal foxes wake.


Next, we group the lineitems by values of `returnflag` and `linestatus` attributes.

In [7]:
@query(
    lineitem
    :filter(shipdate <= date("1998-12-01") - DELTA*days)
    :group(returnflag, linestatus),
    DELTA=90)

4x3 DataFrames.DataFrame
| Row | returnflag | linestatus |
|-----|------------|------------|
| 1   | "A"        | "F"        |
| 2   | "N"        | "F"        |
| 3   | "N"        | "O"        |
| 4   | "R"        | "F"        |

| Row | lineitem                                                                                                                                                                                                                                                                                                                                                        |
|-----|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1   | 11958x16 DataFrames.DataFrame
| Row   | order                | part                                 |
|-------|----------------------|--------------------------------------|
| 1     | "Customer#000000988" | "blue drab lime khaki sienna"        |
⋮
| 11957 | "Customer#000000529" | "chiffon royal lime almond midnight" |
| 11958 | "Customer#000001036" | "medium forest blue ghost black"     |

| Row   | supplier             | linenumber | quantity | extendedprice |
|-------|----------------------|------------|----------|---------------|
| 1     | "Supplier#000000053" | 3          | 27       | 25083.54      |
⋮
| 11957 | "Supplier#000000034" | 4          | 14       | 24960.32      |
| 11958 | "Supplier#000000063" | 2          | 42       | 38724.84      |

| Row   | discount | tax  | returnflag | linestatus | shipdate   | commitdate |
|-------|----------|------|------------|------------|------------|------------|
| 1     | 0.06     | 0.07 | "A"        | "F"        | 1994-01-16 | 1993-11-22 |
⋮
| 11957 | 0.08     | 0.07 | "A"        | "F"        | 1994-12-10 | 1994-10-07 |
| 11958 | 0.05     | 0.02 | "A"        | "F"        | 1993-12-13 | 1993-09-29 |

| Row   | receiptdate | shipinstruct        | shipmode | comment            |
|-------|-------------|---------------------|----------|--------------------|
| 1     | 1994-01-23  | "DELIVER IN PERSON" | "SHIP"   | "nal foxes wake. " |
⋮
| 11957 | 1994-12-27  | "TAKE BACK RETURN"  | "TRUCK"  | " express reques"  |
| 11958 | 1993-12-26  | "COLLECT COD"       | "SHIP"   | "tions. daringl"   |                                                                                                                                                                                                                                                                                   |
| 2   | 288x16 DataFrames.DataFrame
| Row | order                | part                                         |
|-----|----------------------|----------------------------------------------|
| 1   | "Customer#000000262" | "hot midnight orchid dim steel"              |
⋮
| 287 | "Customer#000000397" | "wheat blush forest metallic navajo"         |
| 288 | "Customer#000000397" | "burnished seashell floral moccasin antique" |

| Row | supplier             | linenumber | quantity | extendedprice |
|-----|----------------------|------------|----------|---------------|
| 1   | "Supplier#000000027" | 4          | 25       | 26103.50      |
⋮
| 287 | "Supplier#000000045" | 1          | 2        | 2048.24       |
| 288 | "Supplier#000000075" | 2          | 50       | 85740.50      |

| Row | discount | tax  | returnflag | linestatus | shipdate   | commitdate |
|-----|----------|------|------------|------------|------------|------------|
| 1   | 0.04     | 0.01 | "N"        | "F"        | 1995-06-13 | 1995-05-23 |
⋮
| 287 | 0.04     | 0.0  | "N"        | "F"        | 1995-06-17 | 1995-07-08 |
| 288 | 0.02     | 0.01 | "N"        | "F"        | 1995-05-28 | 1995-08-03 |

| Row | receiptdate | ship

For each distinct combination of `returnflag` and `linestatus` values, we see a collection of corresponding lineitem rows.  We can now use aggregate functions to summarize information about matching lineitems.

For example, to find the total number of matching lineitems, we can use `count` aggregate.

In [8]:
@query(
    lineitem
    :filter(shipdate <= date("1998-12-01") - DELTA*days)
    :group(returnflag, linestatus)
    :select(returnflag, linestatus, count(lineitem)),
    DELTA=90)

,returnflag,linestatus,
1,A,F,11958
2,N,F,288
3,N,O,23326
4,R,F,11963


The query output now has the shape of the report.  We only need to calculate the respective fields.

In [9]:
@query(
    lineitem
    :filter(shipdate <= date("1998-12-01") - DELTA*days)
    :group(returnflag, linestatus)
    :select(
        returnflag,
        linestatus,
        sum_qty => sum(lineitem.quantity),
        sum_base_price => sum(lineitem.extendedprice),
        sum_disc_price => sum(lineitem.(extendedprice*(1-discount))),
        sum_charge => sum(lineitem.(extendedprice*(1-discount)*(1+tax))),
        avg_qty => mean(lineitem.quantity),
        avg_price => mean(lineitem.extendedprice),
        avg_disc => mean(lineitem.discount),
        count_order => count(lineitem)),
    DELTA=90)

,returnflag,linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
1,A,F,303771,401515429.00,381465877.43,396851569.64,25.40316106372303,33577.14,0.05011958521491882,11958
2,N,F,7354,9721375.74,9247800.68,9616674.43,25.53472222222222,33754.78,0.04809027777777776,288
3,N,O,593340,787065026.86,748034019.78,777970207.05,25.436851581925747,33741.96,0.04988510674783497,23326
4,R,F,306623,406030555.01,385792652.67,401431970.22,25.630945415029675,33940.53,0.04973083674663541,11963


## Minimum Cost Supplier Query (Q2)
This query finds which supplier should be selected to place an order for a given part in a given region.

### Business Question
The Minimum Cost Supplier Query finds, in a given region, for each part of a certain type and size, the supplier who
can supply it at minimum cost. If several suppliers in that region offer the desired part type and size at the same
(minimum) cost, the query lists the parts from suppliers with the 100 highest account balances. For each supplier,
the query lists the supplier's account balance, name and nation; the part's number and manufacturer; the supplier's
address, phone number and comment information.


### Query Definition
```
select
    s_acctbal,
    s_name,
    n_name,
    p_partkey,
    p_mfgr,
    s_address,
    s_phone,
    s_comment
from
    part,
    supplier,
    partsupp,
    nation,
    region
where
    p_partkey = ps_partkey
    and s_suppkey = ps_suppkey
    and p_size = [SIZE]
    and p_type like '%[TYPE]'
    and s_nationkey = n_nationkey
    and n_regionkey = r_regionkey
    and r_name = '[REGION]'
    and ps_supplycost = (
        select
            min(ps_supplycost)
        from
            partsupp, supplier,
            nation, region
        where
            p_partkey = ps_partkey
            and s_suppkey = ps_suppkey
            and s_nationkey = n_nationkey
            and n_regionkey = r_regionkey
            and r_name = '[REGION]')
order by
    s_acctbal desc,
    n_name,
    s_name,
    p_partkey;
```

### Substitution Parameters
1. SIZE = 8;
2. TYPE = BRASS;
3. REGION = EUROPE.

This report extracts data from tables `part`, `supplier` as well as table `partsupp` that relates each supplier to their inventory of parts.  We start with getting a list of parts of the given size and type in the inventories of all suppliers.

In [10]:
@query(
    partsupp
    :filter(
        (part.size == SIZE) &
        contains(part.type_, TYPE)),
    SIZE=8, TYPE="BRASS")

,part,supplier,availqty,supplycost,comment
1,drab aquamarine red papaya pale,Supplier#000000009,7731,65.54,ronic accounts haggle blithely across the blithely special platelets. furiously final ideas haggle carefully after the fluffily unusual dep
2,drab aquamarine red papaya pale,Supplier#000000032,8847,935.64,"ular, regular requests affix ironically enticing ideas. fur"
3,drab aquamarine red papaya pale,Supplier#000000055,6329,148.79,enticingly final ideas sleep fluffily. blithely final ideas boost fluffily ironic asymptote
4,drab aquamarine red papaya pale,Supplier#000000078,8054,425.78,"the express accounts. even, ironic pinto beans wake furious"
5,blue gainsboro sky burnished puff,Supplier#000000014,8029,765.35,ous packages sleep slyly across the quickly ironic accounts. regular packages thrash above the furiously final theodolites. furi
6,blue gainsboro sky burnished puff,Supplier#000000040,1481,319.99,"coys against the even instructions cajole furiously bold, daring courts. slyly ironic deposits ought to wake blithely regular ideas. slyly regular ideas according to the slyly express ac"
7,blue gainsboro sky burnished puff,Supplier#000000066,718,634.58,furiously slyly ironic packages. pending requests use ruthlessly across the blithely bold deposits. furiously fin
8,blue gainsboro sky burnished puff,Supplier#000000012,2818,237.16,the carefully special frays. final excuses haggle slyly. furiously regular deposits across t
9,plum indian cornflower frosted purple,Supplier#000000073,1076,844.74,"y ironic instructions. even, final requests are. bold instructions use silent, regular accounts-- slyly final excuses doubt fluffily after the quickly p"
10,plum indian cornflower frosted purple,Supplier#000000022,9272,904.83,"ly regular packages. ruthless instructions haggle carefully final, special courts. silently ruthless courts hinder slyly regular packages; regular, regular pac"


Among all suppliers, we are only interested in those who are situated in the given REGION.

In [11]:
@query(
    partsupp
    :filter(
        (part.size == SIZE) &
        contains(part.type_, TYPE) &
        (supplier.nation.region.name == REGION)),
    SIZE=8, TYPE="BRASS", REGION="EUROPE")

,part,supplier,availqty,supplycost,comment
1,drab aquamarine red papaya pale,Supplier#000000032,8847,935.64,"ular, regular requests affix ironically enticing ideas. fur"
2,blue gainsboro sky burnished puff,Supplier#000000040,1481,319.99,"coys against the even instructions cajole furiously bold, daring courts. slyly ironic deposits ought to wake blithely regular ideas. slyly regular ideas according to the slyly express ac"
3,blue gainsboro sky burnished puff,Supplier#000000066,718,634.58,furiously slyly ironic packages. pending requests use ruthlessly across the blithely bold deposits. furiously fin
4,tan ghost cyan salmon goldenrod,Supplier#000000066,322,104.62,g the quickly express dolphins. slyly even instructions nag furiously. carefully final ideas are idly above the ironic asymptotes. unusual requests haggle about t
5,tan ghost cyan salmon goldenrod,Supplier#000000016,6830,298.77,"ss orbits. furiously regular deposits haggle blithely carefully final pinto beans. ironic, silent accounts thrash quickly about the accounts. iron"
6,frosted blue ghost purple coral,Supplier#000000025,290,774.80,"r pinto beans-- quickly final instructions integrate alongside of the sly, silent th"
7,burlywood seashell slate white cream,Supplier#000000040,9737,799.94,ns cajole idly. deposits are quickly blithely even deposits. quickly regular packages boost carefully. blithely ironic asympto
8,burlywood seashell slate white cream,Supplier#000000077,9194,240.78,l requests cajole carefully. blithely express gifts


Some parts are offered by more than one supplier.  We can use combinator `and_around(part)` to relate each row of the output to all the others that provide the same part.

In [12]:
@query(
    partsupp
    :filter(
        (part.size == SIZE) &
        contains(part.type_, TYPE) &
        (supplier.nation.region.name == REGION))
    :select(part, supplier, supplycost, othersupplycost => and_around(part).supplycost),
    SIZE=8, TYPE="BRASS", REGION="EUROPE")

,part,supplier,supplycost,othersupplycost
1,drab aquamarine red papaya pale,Supplier#000000032,935.64,[935.64]
2,blue gainsboro sky burnished puff,Supplier#000000040,319.99,"[319.99,634.58]"
3,blue gainsboro sky burnished puff,Supplier#000000066,634.58,"[319.99,634.58]"
4,tan ghost cyan salmon goldenrod,Supplier#000000066,104.62,"[104.62,298.77]"
5,tan ghost cyan salmon goldenrod,Supplier#000000016,298.77,"[104.62,298.77]"
6,frosted blue ghost purple coral,Supplier#000000025,774.80,[774.80]
7,burlywood seashell slate white cream,Supplier#000000040,799.94,"[799.94,240.78]"
8,burlywood seashell slate white cream,Supplier#000000077,240.78,"[799.94,240.78]"


Similarly, we can select rows that provide their part at the minimal cost.

In [13]:
@query(
    partsupp
    :filter(
        (part.size == SIZE) &
        contains(part.type_, TYPE) &
        (supplier.nation.region.name == REGION))
    :filter(supplycost == min(and_around(part).supplycost)),
    SIZE=8, TYPE="BRASS", REGION="EUROPE")

,part,supplier,availqty,supplycost,comment
1,drab aquamarine red papaya pale,Supplier#000000032,8847,935.64,"ular, regular requests affix ironically enticing ideas. fur"
2,blue gainsboro sky burnished puff,Supplier#000000040,1481,319.99,"coys against the even instructions cajole furiously bold, daring courts. slyly ironic deposits ought to wake blithely regular ideas. slyly regular ideas according to the slyly express ac"
3,tan ghost cyan salmon goldenrod,Supplier#000000066,322,104.62,g the quickly express dolphins. slyly even instructions nag furiously. carefully final ideas are idly above the ironic asymptotes. unusual requests haggle about t
4,frosted blue ghost purple coral,Supplier#000000025,290,774.80,"r pinto beans-- quickly final instructions integrate alongside of the sly, silent th"
5,burlywood seashell slate white cream,Supplier#000000077,9194,240.78,l requests cajole carefully. blithely express gifts


We have the rows that we need in the requested report.  We only need to sort them and show the requested columns.

In [14]:
@query(
    partsupp
    :filter(
        (part.size == SIZE) &
        contains(part.type_, TYPE) &
        (supplier.nation.region.name == REGION))
    :filter(supplycost == min(and_around(part).supplycost))
    :sort(
        supplier.acctbal:desc,
        supplier.nation.name,
        part.name,
        part.id)
    :select(
        supplier.acctbal,
        supplier.name,
        supplier.nation,
        part.id,
        part.mfgr,
        supplier.address,
        supplier.phone,
        supplier.comment),
    SIZE=8, TYPE="BRASS", REGION="EUROPE")

,acctbal,name,nation,id,mfgr,address,phone,comment
1,9198.31,Supplier#000000025,RUSSIA,1156,Manufacturer#5,RCQKONXMFnrodzz6w7fObFVV6CUm2q,32-431-945-3541,ely regular deposits. carefully regular sauternes engage furiously above the regular accounts. idly
2,4186.95,Supplier#000000077,GERMANY,1362,Manufacturer#1,"wVtcr0uH3CyrSiWMLsqnB09Syo,UuZxPMeBghlY",17-281-345-4863,the slyly final asymptotes. blithely pending theodoli
3,3556.47,Supplier#000000032,UNITED KINGDOM,248,Manufacturer#1,yvoD3TtZSx1skQNCK8agk5bZlZLug,33-484-637-7873,usly even depths. quickly ironic theodolites s
4,2455.98,Supplier#000000066,UNITED KINGDOM,805,Manufacturer#5,"qYdruFJQJYYiKvnNVmYfCVydVB8bcW,AW,U6SOV3",33-300-836-9529,ar requests. express orbits de
5,-290.06,Supplier#000000040,RUSSIA,493,Manufacturer#4,zyIeWzbbpkTV37vm1nmSGBxSgd2Kp,32-231-247-6991,final patterns. accounts haggle idly pas
